In [3]:
# import glob # For getting file names
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
# import seaborn as sns
# import torch

from collections import Counter
# from gap_statistic import OptimalK
from scipy import stats, cluster
from sklearn.cluster import KMeans
# from yellowbrick.cluster import KElbowVisualizer

from utils.clustering_utils import *
from utils.conformal_utils import *

%load_ext autoreload
%autoreload 2

Input:

calibration data
--> number of classes $K$
--> number of examples per class (may vary). Let $n_k$ denote the number of total calibration points for clas $k$ and let $n_{min}$ denote the number of examples that the rarest class has

## 1) Generate candidate parameter values 

**Option 1**: Use heuristic based on $K$ and $n_{min}$
Assume $n_{min}>=5$


O($1/\epsilon^2$) samples are needed to distinguish between two distributions that have TV distance of $\epsilon$ [Source:https://cstheory.stackexchange.com/questions/37607/reference-for-the-number-of-samples-needed-to-distinguish-two-probability-distri] 

If we assume that 5 samples are enough to distinguish between 2 clusters. Suppose that the distance between the distributions is $d$
then we need XX samples to distinguish between 3 clusters. If we make the naive assumption that when there are 4 clusters, the TV distance between clusters is 1/2 what it was before ($d/2$), then we need 4 times as many samples as before... Which is 20. That's a lot. I think the naive assumption is also not valid. 

What if we instead assume that the TV distance halves when we double the number of clusters? This assumption also seems unfounded. Honestly, the best thing to do is probably run some simulations... But I should still plug in something adhoc for now. 


First time determining optimal parameters: only test n_clustering < 1/2 of n_total car

Test null of one hypothesis against best parameter combination from the previous step. If we reject the null, we will try a second time to determine optimal parameters, this time allowing n_clustering to be any value up to n_totalcal. Is this overly complicated? 

Second time determining optimal parameters: 


In [ ]:
def get_clustering_parameters_v1(totalcal_labels):
    '''
    Returns a guess of good values for num_clusters and n_clustering.
    
    Output 
    '''
    n = Counter(totalcal_labels).values() # Unordered
    n_min = min(n)
    K = len(n)
    
    sum_n = len(totalcal_labels)
    
    # TODO
    
    return num_clusters, n_clustering
    
    
def get_clustering_parameters_v2():
    '''
    [WIP]
    
    Actually run the clustering, evaluate the gap statistic to choose the best number of clusters.
    Still not clear how to choose n_clustering though. For now, I somewhat arbitrarily choose the
    n_clustering with the smallest gap statistic at the optimal k. I am not sure that it makes 
    sense to compare the gap statistic across different n_clusterings since the underlying data 
    is changing 
    '''
     # TODO
    pass

    return num_clusters, n_clustering


In [ ]:
# TODO: run some synthetic experiments to get a sense of how many samples you need to distinguish between a certain number of distributions

## 2) Test null hypothesis that there is one cluster 

In [ ]:
pval_threshold = .05 

# TODO: update arguments to function call 
pval = test_one_cluster_null(scores, labels, num_classes, num_clusters, 
                            num_trials=100, seed=0, print_results=True)

## 3) If null hypothesis is rejected, run clustered conformal.
Else, run standard conformal on data not used for clustering 

In [ ]:
if pval < pval_threshold: 
    print(f'p={pval} for one cluster null hypothesis, so running Clustered Conformal')
    # Run clustered conformal and return prediction sets
    qhats, preds, coverage_metrics, set_size_metrics = clustered_conformal(totalcal_scores_all, totalcal_labels,
                                                                alpha,
                                                                n_clustering, num_clusters,
                                                                val_scores_all=val_scores_all, val_labels=val_labels)
    
else:
    print(f'p={pval} for one cluster null hypothesis, so running Standard Conformal')
    # Run Standard Conformal and return prediction sets 
    standard_qhat = compute_qhat(totalcal_scores_all, totalcal_labels, alpha=alpha)
    standard_preds = create_prediction_sets(val_scores_all, vanilla_qhat)
    
    class_cov_metrics, set_size_metrics = compute_all_metrics(val_labels, preds, alpha)

# Test

In [7]:
alpha = .1
n_totalcal = 10 # Total number of calibration points (= # clustering examples + # conformal calibration examples)


# Enron - BERT (n_train=500)
softmax_path = "../class-conditional-conformal-datasets/notebooks/.cache/email_softmax_bert_ntrain=500.npy"
labels_path = "../class-conditional-conformal-datasets/notebooks/.cache/email_labels_bert_ntrain=500.npy"

save_folder = f'.cache/enron_n={n_totalcal}'

In [5]:
# Load data
print('Loading softmax scores and labels...')

softmax_scores = np.load(softmax_path)
labels = np.load(labels_path)

num_classes = labels.max() + 1

Loading softmax scores and labels...


In [6]:
scores_all = 1 - softmax_scores


In [8]:
totalcal_scores_all, totalcal_labels, val_scores_all, val_labels = split_X_and_y(scores_all, labels, n_totalcal, num_classes=num_classes, seed=0)

In [13]:
Counter(totalcal_labels).values()

dict_values([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])